In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

# Parameter provided manually.
alert_id = "SA-AG-01"

# Parameters from the alert yaml.
label = "Voltage-gated sodium channel"
region = "2RL:63,000,000-65,000,000"
ir_candidate_genes = [
    dict(
        identifier="AGAP004707",
        names=["Vgsc", "Para"],
        notes="Blah blah blah.",
        references=[
            dict(
                citation="Clarkson et al. (2021)",
                title="FOo bar",
                url="https://doi.org/10.1111/mec.15845",
            ),
            dict(
                citation="Martinez-Torres et al. (1998)",
                title="Spam eggs",
                url="https://pubmed.ncbi.nlm.nih.gov/9535162/",
            ),
        ],
    ),
    dict(
        identifier="AGAP029667",
        names=["Foo", "Bar"],
        notes="Blah blah blah.",
        references=[
            dict(
                citation="Clarkson et al. (2021)",
                title="FOo bar",
                url="https://doi.org/10.1111/mec.15845",
            ),
            dict(
                citation="Martinez-Torres et al. (1998)",
                title="Spam eggs",
                url="https://pubmed.ncbi.nlm.nih.gov/9535162/",
            ),
        ],
    ),
]

# Parameters from the workflow config.yaml.
contigs = ["2RL", "3RL", "X"]
cohorts_analysis = "20240924"
analysis_version = "2025.02.13"
dask_scheduler = "single-threaded"

In [ ]:
from pyprojroot import here

root = here()
%run -i {root}/workflow/common/scripts/setup.py
%run -i {root}/workflow/site/scripts/page-setup.py

region_contig, region_span = region.split(":")
region_start, region_stop = region_span.replace(",", "").split("-")

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Alert Foo


In [ ]:
content = f"This alert reports selection signals on Chromosome {region_contig} within the region {region_span} bp."
Markdown(content)

## Selection signals

{term}`Selection signal`s overlapping this {term}`genome region` are shown in the figure below.

In [ ]:
df_signals = load_signals(
    contig=region_contig,
    start=region_start,
    stop=region_stop,
)

gene_labels = dict()
for item in ir_candidate_genes:
    g = item["identifier"]
    gene_labels[g] = " "

plot_signals(
    df=df_signals,
    contig=region_contig,
    x_min=df_signals["span2_pstart"].min() - 50_000,
    x_max=df_signals["span2_pstop"].max() + 50_000,
    gene_labels=gene_labels,
    genes_height=90,
)

## Cohorts affected

Overlapping {term}`selection signal`s are found in the following {term}`cohort`s. 

In [ ]:
cohorts_affected = df_signals["cohort_id"]
gdf_cohorts_affected = (
    gdf_cohorts.set_index("cohort_id").loc[cohorts_affected].reset_index()
)
plot_cohorts_map(gdf_cohorts=gdf_cohorts_affected, zoom=3, url_prefix="../")

In [ ]:
content = ""
for _, row in df_signals.sort_values("cohort_label").iterrows():
    cohort_id = row["cohort_id"]
    cohort_label = row["cohort_label"]
    url = f"../cohort/{cohort_id}.html"
    content += f"* [{cohort_label}]({url})\n"
Markdown(content)

## Insecticide resistance genes

The following {term}`gene`s are found within this {term}`genome region` and may be driving {term}`recent positive selection` based on evidence for an association with {term}`insecticide resistance`.

In [ ]:
content = ""
for gene in ir_candidate_genes:
    gene_id = gene["identifier"]
    url = f"https://vectorbase.org/vectorbase/app/record/gene/{gene_id}"
    content += f"### [{gene_id}]({url})"
    gene_names = gene["names"]
    if gene_names:
        content += " (" + ", ".join([f"*{n}*" for n in gene_names]) + ")"
    content += "\n\n"
    notes = gene["notes"]
    content += f"{notes} See also:\n\n"
    for reference in gene["references"]:
        citation = reference["citation"]
        url = reference["url"]
        title = reference["title"]
        content += f"* [{citation}]({url}) {title}\n"
    content += "\n"

Markdown(content)